# Industry energy demand

### Import necessary modules

In [2]:
# Check if we are running the notebook directly, if so move workspace to parent dir
import sys
import os
currentdir = os.path.abspath(os.getcwd())
if os.path.basename(currentdir) != 'DemandMappingZambia':  
  sys.path.insert(0, os.path.dirname(currentdir))
  os.chdir('..')
  print(f'Move to {os.getcwd()}')

Move to H:\Projekte\DemandMappingZambia


In [3]:
### Activate geospatial_env first

# Numeric
import numpy as np
import pandas as pd
import math

# System
import os
import shutil
from IPython.display import display, Markdown, HTML, FileLink, FileLinks

# Spatial
import geopandas as gpd
import json
import pyproj
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.wkt import dumps, loads
from shapely.ops import nearest_points
from pyproj import CRS
import ogr, gdal, osr
#import fiona


# Mapping / Plotting
from functools import reduce
#import datapane as dp 
#!datapane login --token="9bde41bfbc4ad14119e32086f9f06d2e5db1d5b8"
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import BeautifyIcon
from folium.plugins import HeatMap
import branca.colormap as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

%matplotlib inline

In [4]:
#import geopandas as gpd   # Note that you require geopandas version >= 0.7 that incluse clip see here for installation (https://gis.stackexchange.com/questions/360127/geopandas-0-6-1-installed-instead-of-0-7-0-in-conda-windows-10#)
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
from geojson import Feature, Point, FeatureCollection
import rasterio.fill
from shapely.geometry import shape, mapping
import json
#from earthpy import clip    clip has been deprecated to geopandas
#import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
warnings.filterwarnings('ignore')

#import contextily as ctx
import h3 as h3
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

pd.options.display.float_format = '{:,.4f}'.format

In [5]:
from utils import processing_raster, finalizing_rasters, spatialjoinvectors

### Define directories and dataset names

In [6]:
### Define directories and dataset names
ROOT_DIR = os.path.abspath(os.curdir)
in_path = ROOT_DIR
out_path = ROOT_DIR + "/Outputs"

## admininstrative boundary
admin_path = in_path + "\\"+ 'admin'
admin_name = "Copperbelt.gpkg"
adm_col_name = "ADM1_NAME"          ## Provide the name of the column you want to use to clip the hexagons e.g., "NAME" or "ADM1_NAME"

## mines layer
mines_path = in_path + "/Industry/Data/mines"
# mines_name = 'mines_zambia.gpkg'
mines_name = 'mineral_facilities_zambia_Dec_2023.gpkg'

In [7]:
## Coordinate and projection systems
crs_WGS84 = CRS("EPSG:4326")    # Originan WGS84 coordinate system
crs_proj = CRS("EPSG:32736")    # Projection system for the selected country -- see http://epsg.io/ for more info

### Import layers to be used

In [8]:
hexagons = gpd.read_file(out_path + "\\" + "h3_grid_at_hex_7.shp")

In [9]:
grid = hexagons

In [10]:
## mines
mines = gpd.read_file(mines_path + "/" + mines_name)

In [11]:
## admininstrative boundary
admin_gdf = gpd.read_file(admin_path + "/" + admin_name)

# Part 1. Extract GIS-based attributes

## Extract information from vector layers

Extract sum production of mines in each cluster (hex)

In [12]:
## Run the extraction
#grid.drop(['Commodity Production - tonne (tonnes)'], axis=1, inplace=True) ##uncomment if you want to rerun
# columnNameMines = "Commodity Production - tonne (tonnes)"

#columnNameMines = 'Ore processed (tonnes)'
columnNameMines = 'Energy Elec (TJ)'
#grid, mines = spatialjoinvectors("Mines", columnNameMines, admin_gdf, crs_WGS84, grid, "sum")
grid, mines = spatialjoinvectors("Energy", columnNameMines, admin_gdf, crs_WGS84, grid, "sum")

grid[columnNameMines] = grid[columnNameMines].fillna(0)
grid.head(4)

,hexagons,lon,lat,index_righ,ADM1_NAME,id,geometry,Energy Elec (TJ)
0,875534665ffffff,27.4684,-12.7930,0.0000,COPPERBELT,1,"POLYGON ((27.48116 -12.79734, 27.47111 -12.807...",0.0000
1,875536804ffffff,27.9189,-13.6953,0.0000,COPPERBELT,2,"POLYGON ((27.93173 -13.69992, 27.92157 -13.710...",0.0000
2,875536815ffffff,27.9039,-13.7454,0.0000,COPPERBELT,3,"POLYGON ((27.91667 -13.75003, 27.90651 -13.760...",0.0000
3,87346970effffff,28.3954,-13.8803,0.0000,COPPERBELT,4,"POLYGON ((28.40823 -13.88488, 28.39805 -13.895...",0.0000


In [13]:
grid[columnNameMines].sum()

14951.62962857

# Part 2. Compute demand

In [20]:
# Assess total energy consumption and total production
#elec_nonFerrousMetals = 22897 #PJ UN stats
elec_nonFerrousMetals_TJ = 22897 # TJ from UN stats
elec_mining_TJ = 818 # TJ from UN stats

elec_nonFerrousMetals = elec_nonFerrousMetals_TJ /(3600) *10**6 # (3.6e-6) #conversion in MWh
elec_mining = elec_mining_TJ /(3600) *10**6 #conversion in MWh
#total_production = mines[columnNameMines].str.replace(',', '').astype(float).sum()
total_elec_energy_consum = mines[columnNameMines].sum() # total 

# total_production = sum(grid[columnNameMines]) # ton

#energycons_perton = elec_nonFerrousMetals/total_production # MWh/t
coverage_elec_nonFe_mining = total_elec_energy_consum/(elec_nonFerrousMetals_TJ+elec_mining_TJ)

# print("total production:", f"{total_production/10**3:,.0f}", "kt")
print("total calculated energy consumption:", f"{total_elec_energy_consum:,.1f}", "TJ or ", f"{total_elec_energy_consum/3600:,.1f}", "TWh")

print("Calculated energy in total statistical nonFerrousMetals and mining electricity consumption:", f"{coverage_elec_nonFe_mining*100:,.0f}", "%")

#print("total industry electricity consumption:",f"{elec_nonFerrousMetals/10**6:,.1f}", "TWh")
print("total statistical nonFerrousMetals electricity consumption:",f"{elec_nonFerrousMetals_TJ:,.1f}", "TJ or ", f"{elec_nonFerrousMetals/10**6:,.1f}", "TWh")
print("total statistical mining electricity consumption:",f"{elec_mining_TJ:,.1f}", "TJ or ", f"{elec_mining/10**6:,.1f}", "TWh")
print("total statistical nonFerrousMetals and mining electricity consumption:",f"{elec_nonFerrousMetals_TJ+elec_mining_TJ:,.1f}", "TJ or ", f"{(elec_nonFerrousMetals+elec_mining)/10**6:,.1f}", "TWh")


#print("energy per tonne of ore:", f"{energycons_perton:,.0f}", "MWh/t")

total calculated energy consumption: 23,565.7 TJ or  6.5 TWh
Calculated energy in total statistical nonFerrousMetals and mining electricity consumption: 99 %
total statistical nonFerrousMetals electricity consumption: 22,897.0 TJ or  6.4 TWh
total statistical mining electricity consumption: 818.0 TJ or  0.2 TWh
total statistical nonFerrousMetals and mining electricity consumption: 23,715.0 TJ or  6.6 TWh


In [21]:
#Allocate to each hexagon the industry energy consumption
#grid["IndEnergy"]=grid[columnNameMines]*energycons_perton
grid["IndEnergy"]=grid[columnNameMines]/coverage_elec_nonFe_mining # TJ

grid.head(3)
total_industryenergy = grid["IndEnergy"].sum()
#print("Industry electricity consumption:",f"{total_industryenergy/10**6:,.0f}", "TWh")
print("Industry electricity consumption in region:",f"{total_industryenergy/3600:,.1f}", "TWh")

Industry electricity consumption in region: 4.2 TWh


In [22]:
grid.to_file(out_path + "\\" + 'ind_energy_map.shp', index=False)
grid.head(3)

,hexagons,lon,lat,index_righ,ADM1_NAME,id,geometry,Energy Elec (TJ),IndEnergy
0,875534665ffffff,27.4684,-12.7930,0.0000,COPPERBELT,1,"POLYGON ((27.48116 -12.79734, 27.47111 -12.807...",0.0000,0.0000
1,875536804ffffff,27.9189,-13.6953,0.0000,COPPERBELT,2,"POLYGON ((27.93173 -13.69992, 27.92157 -13.710...",0.0000,0.0000
2,875536815ffffff,27.9039,-13.7454,0.0000,COPPERBELT,3,"POLYGON ((27.91667 -13.75003, 27.90651 -13.760...",0.0000,0.0000


In [23]:
grid[columnNameMines]

0      0.0000
1      0.0000
2      0.0000
3      0.0000
4      0.0000
        ...  
5161   0.0000
5162   0.0000
5163   0.0000
5164   0.0000
5165   0.0000
Name: Energy Elec (TJ), Length: 5166, dtype: float64